In [460]:
import pickle
import sqlite3
import pandas as pd
%pip install category_encoders
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error 

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [397]:
df = pd.read_csv('Coffee_Qlty.csv')
df.head(5)

,REC_ID,Species,Continent.of.Origin,Country.of.Origin,Harvest.Year,Expiration,Variety,Color,Processing.Method,Aroma,...,Acidity,Body,Balance,Uniformity,Clean.Cup,Sweetness,Moisture,Quakers,Category.One.Defects,Category.Two.Defects
0,0,Arabica,Africa,Ethiopia,2014.0,04/03/16,NaN,Green,Washed / Wet,8.67,...,8.75,8.50,8.42,10.0,10.0,10.0,0.12,0,0,0
1,1,Arabica,Africa,Ethiopia,2014.0,04/03/16,Other,Green,Washed / Wet,8.75,...,8.58,8.42,8.42,10.0,10.0,10.0,0.12,0,0,1
2,2,Arabica,North America,Guatemala,NaN,05/31/11,Bourbon,NaN,NaN,8.42,...,8.42,8.33,8.42,10.0,10.0,10.0,0.00,0,0,0
3,3,Arabica,Africa,Ethiopia,2014.0,03/25/16,NaN,Green,Natural / Dry,8.17,...,8.42,8.50,8.25,10.0,10.0,10.0,0.11,0,0,2
4,4,Arabica,Africa,Ethiopia,2014.0,04/03/16,Other,Green,Washed / Wet,8.25,...,8.50,8.42,8.33,10.0,10.0,10.0,0.12,0,0,2


In [398]:
df.dtypes

REC_ID                    int64
Species                  object
Continent.of.Origin      object
Country.of.Origin        object
Harvest.Year            float64
Expiration               object
Variety                  object
Color                    object
Processing.Method        object
Aroma                   float64
Flavor                  float64
Aftertaste              float64
Acidity                 float64
Body                    float64
Balance                 float64
Uniformity              float64
Clean.Cup               float64
Sweetness               float64
Moisture                float64
Quakers                   int64
Category.One.Defects      int64
Category.Two.Defects      int64
dtype: object

In [399]:
print(df.isnull().sum())

REC_ID                    0
Species                   0
Continent.of.Origin       1
Country.of.Origin         1
Harvest.Year             60
Expiration                0
Variety                 226
Color                   270
Processing.Method       170
Aroma                     0
Flavor                    0
Aftertaste                0
Acidity                   0
Body                      0
Balance                   0
Uniformity                0
Clean.Cup                 0
Sweetness                 0
Moisture                  0
Quakers                   0
Category.One.Defects      0
Category.Two.Defects      0
dtype: int64


In [400]:
df = df.dropna()

In [401]:
df.isnull().sum()

REC_ID                  0
Species                 0
Continent.of.Origin     0
Country.of.Origin       0
Harvest.Year            0
Expiration              0
Variety                 0
Color                   0
Processing.Method       0
Aroma                   0
Flavor                  0
Aftertaste              0
Acidity                 0
Body                    0
Balance                 0
Uniformity              0
Clean.Cup               0
Sweetness               0
Moisture                0
Quakers                 0
Category.One.Defects    0
Category.Two.Defects    0
dtype: int64

In [402]:
df.shape

(946, 22)

In [403]:
df.dtypes

REC_ID                    int64
Species                  object
Continent.of.Origin      object
Country.of.Origin        object
Harvest.Year            float64
Expiration               object
Variety                  object
Color                    object
Processing.Method        object
Aroma                   float64
Flavor                  float64
Aftertaste              float64
Acidity                 float64
Body                    float64
Balance                 float64
Uniformity              float64
Clean.Cup               float64
Sweetness               float64
Moisture                float64
Quakers                   int64
Category.One.Defects      int64
Category.Two.Defects      int64
dtype: object

In [404]:
df['Harvest.Year'] = df['Harvest.Year'].astype(int)


In [405]:
df['Harvest.Year']


1       2014
4       2014
6       2012
9       2014
18      2015
        ... 
1308    2016
1309    2012
1319    2015
1322    2015
1333    2012
Name: Harvest.Year, Length: 946, dtype: int32

In [406]:
df['Expiration'] = pd.to_datetime(df['Expiration'], format='%m/%d/%y')
df['Expiration'] = df['Expiration'].dt.year

In [407]:
df['Expiration']

1       2016
4       2016
6       2013
9       2016
18      2017
        ... 
1308    2018
1309    2013
1319    2017
1322    2017
1333    2013
Name: Expiration, Length: 946, dtype: int32

In [408]:
for col in df.columns:
    print(col)
    print(df[col].unique())

REC_ID
[   1    4    6    9   18   21   22   25   27   29   32   33   35   40
   43   44   45   47   48   49   52   53   54   55   57   58   62   66
   67   68   69   71   73   75   76   77   78   79   80   84   85   86
   88   91   92   96   97   98   99  101  102  107  108  109  110  112
  113  115  116  120  121  122  125  126  128  129  130  132  133  135
  136  137  138  139  141  145  146  147  148  149  152  153  154  155
  156  157  158  159  160  161  162  163  164  165  166  168  171  172
  173  174  175  176  179  181  182  184  185  186  191  192  193  194
  195  196  197  198  199  200  201  202  203  204  205  206  207  208
  209  210  211  212  213  214  215  216  217  219  220  221  222  224
  225  226  227  228  229  230  231  232  233  234  236  237  238  241
  242  243  244  245  246  248  249  250  251  252  253  254  255  256
  258  259  260  261  262  268  269  270  271  273  274  276  277  279
  280  282  283  284  285  287  288  289  291  296  297  298  300  301

In [409]:
df = df.drop('REC_ID', axis=1)

In [410]:
df.columns

Index(['Species', 'Continent.of.Origin', 'Country.of.Origin', 'Harvest.Year',
       'Expiration', 'Variety', 'Color', 'Processing.Method', 'Aroma',
       'Flavor', 'Aftertaste', 'Acidity', 'Body', 'Balance', 'Uniformity',
       'Clean.Cup', 'Sweetness', 'Moisture', 'Quakers', 'Category.One.Defects',
       'Category.Two.Defects'],
      dtype='object')

In [411]:
df['Defective'] = df['Category.One.Defects'] + df['Category.Two.Defects']
df['Defective'] = df['Defective'].astype(int)
df['Defective']


1       1
4       2
6       0
9       4
18      0
       ..
1308    6
1309    4
1319    0
1322    0
1333    0
Name: Defective, Length: 946, dtype: int32

In [412]:
df['Color'] = df['Color'].replace(['Bluish-Green', 'Blue-Green'], 'Bluish-green')
df.Color.unique()

array(['Green', 'Bluish-green'], dtype=object)

In [413]:
import category_encoders as ce
encoder = ce.OrdinalEncoder(mapping=[{'col': 'Species', 'mapping': {'Arabica': 1, 'Robusta': 2}}, 
                                    {'col': 'Color', 'mapping': {'Green': 1, 'Bluish-green':2}},
                                     {'col': 'Processing.Method', 'mapping': {'Washed / Wet': 1, 'Natural / Dry': 2, 'Pulped natural / honey': 3, 'Semi-washed / Semi-pulped': 4, 'Other': 5}},
                                     {'col': 'Continent.of.Origin', 'mapping': {'Africa': 1, 'South America': 2, 'Asia': 3, 'North America': 4}},
                                     {'col': 'Country.of.Origin', 'mapping': {'Ethiopia': 1, 'Peru': 2, 'China': 3, 'Costa Rica': 4, 'Mexico': 5, 'Brazil': 6, 'Uganda': 7, 'Taiwan': 8, 'Kenya': 9, 'Thailand': 10, 'Colombia': 11, 'Guatemala': 12, 'Panama': 13, 'El Salvador': 14, 'Indonesia': 15, 'United States': 16, 'Tanzania': 17, 'Honduras': 18, 'Nicaragua': 19, 'Haiti': 20, 'Vietnam': 21, 'Philippines': 22, 'Rwanda': 23, 'Malawi': 24, 'Laos': 25, 'Myanmar': 26, 'India': 27}
}
                                    ])
encoder.fit(df)
df = encoder.transform(df)
df.Species

1       1
4       1
6       1
9       1
18      1
       ..
1308    1
1309    1
1319    2
1322    2
1333    2
Name: Species, Length: 946, dtype: int32

In [414]:
conn = sqlite3.connect('majordb.db')

df.to_sql('coffee', conn, if_exists='replace', index=False)
conn.commit()
conn.close()

In [461]:
features = ['Species','Continent.of.Origin','Harvest.Year', 'Aroma', 'Flavor', 'Acidity', 'Uniformity','Processing.Method', 'Country.of.Origin','Moisture', 'Balance','Clean.Cup']
y = df['Defective']
x = df[features]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

rf = RandomForestRegressor(n_estimators=100, random_state=1)
rf.fit(X_train,y_train)

y_pred = rf.predict(X_test)
mean_squared_error(y_test, y_pred)


31.276183230994157

In [416]:
pickle.dump(rf, open('regression_model.pkl', 'wb'))

In [417]:
val_score = rf.score(X_test, y_test)
print(f"Validation R^2 for noisy model is: {val_score}")
print(rf.oob_score_)

Validation R^2 for noisy model is: 0.2170010744903389
0.07071878780761354


In [418]:
X = df[['Species','Color' , 'Country.of.Origin' , 'Harvest.Year']]
Y = df[['Continent.of.Origin']]


x_train, x_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
clf = DecisionTreeClassifier(random_state=42)

clf.fit(x_train, Y_train)


DecisionTreeClassifier(random_state=42)

In [419]:
pickle.dump(clf, open('classification_model.pkl', 'wb'))

In [442]:
Y_pred = clf.predict(x_test)

# Evaluate model performance
accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy:", accuracy)

# Print classification report
print("Classification Report:")
print(classification_report(Y_test, Y_pred))

    # Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(Y_test, Y_pred))

Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        18
           2       1.00      1.00      1.00        48
           3       1.00      1.00      1.00        26
           4       1.00      1.00      1.00        98

    accuracy                           1.00       190
   macro avg       1.00      1.00      1.00       190
weighted avg       1.00      1.00      1.00       190

Confusion Matrix:
[[18  0  0  0]
 [ 0 48  0  0]
 [ 0  0 26  0]
 [ 0  0  0 98]]
